# Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Widgets

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("storageName", "adlsprojectsmartdata")
dbutils.widgets.text("container", "raw")
dbutils.widgets.text("catalog", "project_smartdata")
dbutils.widgets.text("schema", "bronze")

# Constants

In [0]:
storage_name = dbutils.widgets.get("storageName")
container = dbutils.widgets.get("container")
catalog =  dbutils.widgets.get("catalog")
schema =  dbutils.widgets.get("schema")

# Paths

In [0]:
path_base = f"abfss://{container}@{storage_name}.dfs.core.windows.net/"
path_sales =  f"{path_base}sales.csv"

# Structure

In [0]:
sales_schema =  StructType(fields=[
    StructField("sale_id", StringType(), False),
    StructField("sale_date", StringType(), False),
    StructField("store_id", StringType(), True),
    StructField("product_id", StringType(), False),
    StructField("quantity", IntegerType(), True)
])

# Read source

In [0]:
df_sales =  spark.read.option("header", True)\
                      .schema(sales_schema)\
                      .csv(path_sales)

In [0]:
df_sales_final =  df_sales.withColumn("processed_at", current_timestamp())

# Save

In [0]:
df_sales_final.write.mode("overwrite").saveAsTable(f"{catalog}.{schema}.sales")